In [4]:
#1 Реализация LineSearchTool
import numpy as np
from numpy.linalg import LinAlgError
import scipy
from scipy.optimize import linesearch
from datetime import datetime
from collections import defaultdict


class LineSearchTool(object):
   
    def __init__(self, method='Wolfe', **kwargs):
        self._method = method
        if self._method == 'Wolfe':
            self.c1 = kwargs.get('c1', 1e-4)
            self.c2 = kwargs.get('c2', 0.9)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Armijo':
            self.c1 = kwargs.get('c1', 1e-4)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Constant':
            self.c = kwargs.get('c', 1.0)
        else:
            raise ValueError('Unknown method {}'.format(method))

    @classmethod
    def from_dict(cls, options):
        if type(options) != dict:
            raise TypeError('LineSearchTool initializer must be of type dict')
        return cls(**options)

    def to_dict(self):
        return self.__dict__

    def line_search(self, oracle, x_k, d_k, previous_alpha=None):
 
        phi0 = oracle.func_directional(x_k, d_k, 0)
        dphi0 = oracle.grad_directional(x_k, d_k, 0)
        
        # Проверка, что направление является направлением спуска
        if dphi0 >= 0:
            return None
        
        if self._method == 'Constant':
            return self.c
        
        elif self._method == 'Armijo':
            # Адаптивный метод подбора шага
            if previous_alpha is not None:
                alpha = previous_alpha * 2
            else:
                alpha = self.alpha_0
                
            # Метод дробления шага (бэктрекинг)
            while oracle.func_directional(x_k, d_k, alpha) > phi0 + self.c1 * alpha * dphi0:
                alpha *= 0.5
                if alpha < 1e-12:  # Предотвращение бесконечного цикла
                    return None
            return alpha
        
        elif self._method == 'Wolfe':
            # Используем библиотечную реализацию Wolfe условий
            try:
                alpha, _, _, _ = linesearch.scalar_search_wolfe2(
                    phi=lambda alpha: oracle.func_directional(x_k, d_k, alpha),
                    derphi=lambda alpha: oracle.grad_directional(x_k, d_k, alpha),
                    phi0=phi0,
                    derphi0=dphi0,
                    c1=self.c1,
                    c2=self.c2
                )
                
                if alpha is None:
                    # Если Wolfe не сработал, используем Armijo с alpha_0
                    alpha = self.alpha_0
                    while oracle.func_directional(x_k, d_k, alpha) > phi0 + self.c1 * alpha * dphi0:
                        alpha *= 0.5
                        if alpha < 1e-12:
                            return None
                return alpha
            except Exception:
                # В случае ошибки используем Armijo
                alpha = self.alpha_0
                while oracle.func_directional(x_k, d_k, alpha) > phi0 + self.c1 * alpha * dphi0:
                    alpha *= 0.5
                    if alpha < 1e-12:
                        return None
                return alpha

#2 Реализация Gradient Descent
def get_line_search_tool(line_search_options=None):
    if line_search_options:
        if type(line_search_options) is LineSearchTool:
            return line_search_options
        else:
            return LineSearchTool.from_dict(line_search_options)
    else:
        return LineSearchTool()


def gradient_descent(oracle, x_0, tolerance=1e-5, max_iter=10000,
                     line_search_options=None, trace=False, display=False):
    
    history = defaultdict(list) if trace else None
    line_search_tool = get_line_search_tool(line_search_options)
    x_k = np.copy(x_0)
    
    # Вычисляем начальный градиент и его норму для критерия остановки
    grad_k = oracle.grad(x_k)
    grad_norm_2 = np.linalg.norm(grad_k) ** 2
    grad_norm_0_2 = grad_norm_2
    
    if trace:
        history['time'].append(0.0)
        history['func'].append(oracle.func(x_k))
        history['grad_norm'].append(np.sqrt(grad_norm_2))
        if x_k.size <= 2:
            history['x'].append(x_k.copy())
    
    start_time = datetime.now()
    previous_alpha = None
    
    for iteration in range(max_iter):
        # Критерий остановки (1.1): ||∇f(x_k)||^2 ≤ ε * ||∇f(x_0)||^2
        if grad_norm_2 <= tolerance * grad_norm_0_2:
            return x_k, 'success', history
        
        # Направление спуска - антиградиент
        d_k = -grad_k
        
        # Линейный поиск
        try:
            alpha = line_search_tool.line_search(oracle, x_k, d_k, previous_alpha)
            if alpha is None:
                if display:
                    print(f"Iteration {iteration}: line search failed")
                return x_k, 'computational_error', history
            previous_alpha = alpha
        except Exception as e:
            if display:
                print(f"Iteration {iteration}: computational error - {e}")
            return x_k, 'computational_error', history
        
        # Обновление точки
        x_k = x_k + alpha * d_k
        
        # Вычисление градиента в новой точке
        try:
            grad_k = oracle.grad(x_k)
            grad_norm_2 = np.linalg.norm(grad_k) ** 2
        except Exception as e:
            if display:
                print(f"Iteration {iteration}: computational error in gradient - {e}")
            return x_k, 'computational_error', history
        
        # Сохранение информации для трассировки
        if trace:
            current_time = (datetime.now() - start_time).total_seconds()
            history['time'].append(current_time)
            history['func'].append(oracle.func(x_k))
            history['grad_norm'].append(np.sqrt(grad_norm_2))
            if x_k.size <= 2:
                history['x'].append(x_k.copy())
        
        if display and (iteration + 1) % 100 == 0:
            print(f"Iteration {iteration + 1}: f = {history['func'][-1]:.6f}, "
                  f"||grad||^2 = {grad_norm_2:.6e}, alpha = {alpha:.6f}")
    
    # Превышено максимальное число итераций
    return x_k, 'iterations_exceeded', history

#3 Реализация Newton Method
def newton(oracle, x_0, tolerance=1e-5, max_iter=100,
           line_search_options=None, trace=False, display=False):
    
    history = defaultdict(list) if trace else None
    line_search_tool = get_line_search_tool(line_search_options)
    x_k = np.copy(x_0)
    
    # Вычисляем начальный градиент и его норму для критерия остановки
    try:
        grad_k = oracle.grad(x_k)
        grad_norm_2 = np.linalg.norm(grad_k) ** 2
        grad_norm_0_2 = grad_norm_2
    except Exception as e:
        if display:
            print(f"Initial point: computational error in gradient - {e}")
        return x_k, 'computational_error', history
    
    if trace:
        history['time'].append(0.0)
        history['func'].append(oracle.func(x_k))
        history['grad_norm'].append(np.sqrt(grad_norm_2))
        if x_k.size <= 2:
            history['x'].append(x_k.copy())
    
    start_time = datetime.now()
    
    for iteration in range(max_iter):
        # Критерий остановки (1.1): ||∇f(x_k)||^2 ≤ ε * ||∇f(x_0)||^2
        if grad_norm_2 <= tolerance * grad_norm_0_2:
            return x_k, 'success', history
        
        # Вычисление гессиана и решение системы ∇^2 f(x_k) d_k = -∇f(x_k)
        try:
            hess_k = oracle.hess(x_k)
            # Используем разложение Холецкого для положительно определенных матриц
            try:
                cho_factor = scipy.linalg.cho_factor(hess_k)
                d_k = scipy.linalg.cho_solve(cho_factor, -grad_k)
            except LinAlgError:
                # Если матрица не положительно определена, используем общий метод
                d_k = np.linalg.solve(hess_k, -grad_k)
        except Exception as e:
            if display:
                print(f"Iteration {iteration}: Newton direction error - {e}")
            return x_k, 'newton_direction_error', history
        
        # Проверка направления спуска
        dphi0 = grad_k.dot(d_k)
        if dphi0 >= 0:
            if display:
                print(f"Iteration {iteration}: Not a descent direction")
            return x_k, 'newton_direction_error', history
        
        # Линейный поиск (всегда начинаем с alpha = 1.0 для метода Ньютона)
        try:
            # Для метода Ньютона всегда пробуем alpha = 1.0 первым
            if line_search_tool._method == 'Wolfe':
                # Специальная обработка для Wolfe: сначала пробуем alpha=1.0
                phi0 = oracle.func_directional(x_k, d_k, 0)
                dphi0 = oracle.grad_directional(x_k, d_k, 0)
                
                try:
                    alpha, _, _, _ = linesearch.scalar_search_wolfe2(
                        phi=lambda alpha: oracle.func_directional(x_k, d_k, alpha),
                        derphi=lambda alpha: oracle.grad_directional(x_k, d_k, alpha),
                        phi0=phi0,
                        derphi0=dphi0,
                        c1=line_search_tool.c1,
                        c2=line_search_tool.c2,
                        amax=1.0  # Ограничиваем максимальный шаг 1.0
                    )
                    
                    if alpha is None:
                        # Если Wolfe не сработал, используем Armijo с alpha=1.0
                        alpha = 1.0
                        while oracle.func_directional(x_k, d_k, alpha) > phi0 + line_search_tool.c1 * alpha * dphi0:
                            alpha *= 0.5
                            if alpha < 1e-12:
                                alpha = None
                                break
                except Exception:
                    alpha = 1.0
                    while oracle.func_directional(x_k, d_k, alpha) > phi0 + line_search_tool.c1 * alpha * dphi0:
                        alpha *= 0.5
                        if alpha < 1e-12:
                            alpha = None
                            break
            else:
                # Для Armijo и Constant используем стандартную процедуру
                alpha = line_search_tool.line_search(oracle, x_k, d_k, previous_alpha=1.0)
            
            if alpha is None:
                if display:
                    print(f"Iteration {iteration}: line search failed")
                return x_k, 'computational_error', history
        except Exception as e:
            if display:
                print(f"Iteration {iteration}: computational error in line search - {e}")
            return x_k, 'computational_error', history
        
        # Обновление точки
        x_k = x_k + alpha * d_k
        
        # Вычисление градиента в новой точке
        try:
            grad_k = oracle.grad(x_k)
            grad_norm_2 = np.linalg.norm(grad_k) ** 2
        except Exception as e:
            if display:
                print(f"Iteration {iteration}: computational error in gradient - {e}")
            return x_k, 'computational_error', history
        
        # Сохранение информации для трассировки
        if trace:
            current_time = (datetime.now() - start_time).total_seconds()
            history['time'].append(current_time)
            history['func'].append(oracle.func(x_k))
            history['grad_norm'].append(np.sqrt(grad_norm_2))
            if x_k.size <= 2:
                history['x'].append(x_k.copy())
        
        if display and (iteration + 1) % 10 == 0:
            print(f"Iteration {iteration + 1}: f = {history['func'][-1]:.6f}, "
                  f"||grad||^2 = {grad_norm_2:.6e}, alpha = {alpha:.6f}")
    
    # Превышено максимальное число итераций
    return x_k, 'iterations_exceeded', history

#4 Реализация LogRegL2Oracle

import numpy as np
import scipy
from scipy.special import expit


class BaseSmoothOracle(object):
    """
    Base class for implementation of oracles.
    """
    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')
    
    def hess(self, x):
        """
        Computes the Hessian matrix at point x.
        """
        raise NotImplementedError('Hessian oracle is not implemented.')
    
    def func_directional(self, x, d, alpha):
        """
        Computes phi(alpha) = f(x + alpha*d).
        """
        return np.squeeze(self.func(x + alpha * d))

    def grad_directional(self, x, d, alpha):
        """
        Computes phi'(alpha) = (f(x + alpha*d))'_{alpha}
        """
        return np.squeeze(self.grad(x + alpha * d).dot(d))


class QuadraticOracle(BaseSmoothOracle):
    """
    Oracle for quadratic function:
       func(x) = 1/2 x^TAx - b^Tx.
    """

    def __init__(self, A, b):
        if not scipy.sparse.isspmatrix_dia(A) and not np.allclose(A, A.T):
            raise ValueError('A should be a symmetric matrix.')
        self.A = A
        self.b = b

    def func(self, x):
        return 0.5 * np.dot(self.A.dot(x), x) - self.b.dot(x)

    def grad(self, x):
        return self.A.dot(x) - self.b

    def hess(self, x):
        return self.A 


class LogRegL2Oracle(BaseSmoothOracle):
    
    def __init__(self, matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef):
        self.matvec_Ax = matvec_Ax
        self.matvec_ATx = matvec_ATx
        self.matmat_ATsA = matmat_ATsA
        self.b = b
        self.regcoef = regcoef
        self.m = len(b)

    def func(self, x):
        """
        f(x) = 1/m * sum(log(1 + exp(-b_i * a_i^T x))) + regcoef/2 * ||x||^2
        Векторизованная реализация без циклов.
        """
        Ax = self.matvec_Ax(x)
        # Используем np.logaddexp для численной устойчивости
        # log(1 + exp(-b_i * a_i^T x)) = -logaddexp(0, -b_i * a_i^T x)
        # Но лучше直接用: logaddexp(0, -b_i * a_i^T x)
        log_term = np.logaddexp(0, -self.b * Ax)
        f = np.mean(log_term) + 0.5 * self.regcoef * np.dot(x, x)
        return f

    def grad(self, x):
        """
        ∇f(x) = -1/m * A^T (b / (1 + exp(b * Ax))) + regcoef * x
        Используем scipy.special.expit для сигмоиды.
        """
        Ax = self.matvec_Ax(x)
        # sigma = 1/(1+exp(-z))
        # b / (1 + exp(b * Ax)) = b * sigma(-b * Ax)
        sigma_minus = expit(-self.b * Ax)
        grad_main = -self.matvec_ATx(self.b * sigma_minus) / self.m
        grad_reg = self.regcoef * x
        return grad_main + grad_reg

    def hess(self, x):
        """
        ∇^2 f(x) = 1/m * A^T * diag(sigma(b*Ax) * sigma(-b*Ax)) * A + regcoef * I
        """
        Ax = self.matvec_Ax(x)
        # sigma(b*Ax) * sigma(-b*Ax) - это variance для логистической регрессии
        sigma_pos = expit(self.b * Ax)
        sigma_neg = expit(-self.b * Ax)
        s = sigma_pos * sigma_neg  # Поэлементное произведение
        
        # A^T * Diag(s) * A
        hess_main = self.matmat_ATsA(s) / self.m
        hess_reg = self.regcoef * np.eye(len(x))
        return hess_main + hess_reg


class LogRegL2OptimizedOracle(LogRegL2Oracle):
    
    def __init__(self, matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef):
        super().__init__(matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef)
        self._cached_Ax = None
        self._cached_Ad = None
        self._cached_x = None
        self._cached_d = None
        self._cached_x_alpha = None
        self._cached_Ax_alpha = None

    def func_directional(self, x, d, alpha):
        """
        Оптимизированная версия: предвычисляем Ax и Ad.
        """
        x_alpha = x + alpha * d
        
        # Проверяем кэш для Ax
        if self._cached_x is not None and np.array_equal(x, self._cached_x):
            Ax = self._cached_Ax
        else:
            Ax = self.matvec_Ax(x)
            self._cached_x = x.copy()
            self._cached_Ax = Ax
        
        # Вычисляем Ad, если нужно
        if self._cached_d is not None and np.array_equal(d, self._cached_d):
            Ad = self._cached_Ad
        else:
            Ad = self.matvec_Ax(d)
            self._cached_d = d.copy()
            self._cached_Ad = Ad
        
        # Вычисляем Ax_alpha = Ax + alpha * Ad
        Ax_alpha = Ax + alpha * Ad
        
        # Кэшируем для последующих вызовов в этой точке
        self._cached_x_alpha = x_alpha.copy()
        self._cached_Ax_alpha = Ax_alpha
        
        # Вычисляем значение функции
        log_term = np.logaddexp(0, -self.b * Ax_alpha)
        f = np.mean(log_term) + 0.5 * self.regcoef * np.dot(x_alpha, x_alpha)
        return f

    def grad_directional(self, x, d, alpha):
        """
        Оптимизированная версия: предвычисляем Ax и Ad.
        """
        x_alpha = x + alpha * d
        
        # Проверяем кэш для Ax_alpha
        if (self._cached_x_alpha is not None and 
            np.array_equal(x_alpha, self._cached_x_alpha)):
            Ax_alpha = self._cached_Ax_alpha
        else:
            # Проверяем кэш для Ax
            if self._cached_x is not None and np.array_equal(x, self._cached_x):
                Ax = self._cached_Ax
            else:
                Ax = self.matvec_Ax(x)
                self._cached_x = x.copy()
                self._cached_Ax = Ax
            
            # Проверяем кэш для Ad
            if self._cached_d is not None and np.array_equal(d, self._cached_d):
                Ad = self._cached_Ad
            else:
                Ad = self.matvec_Ax(d)
                self._cached_d = d.copy()
                self._cached_Ad = Ad
            
            Ax_alpha = Ax + alpha * Ad
            self._cached_x_alpha = x_alpha.copy()
            self._cached_Ax_alpha = Ax_alpha
        
        # Вычисляем производную
        sigma_minus = expit(-self.b * Ax_alpha)
        grad_dir = -np.dot(self.b * sigma_minus, Ad) / self.m + self.regcoef * np.dot(x_alpha, d)
        return grad_dir


def create_log_reg_oracle(A, b, regcoef, oracle_type='usual'):
    
    def matvec_Ax(x):
        return A.dot(x)
    
    def matvec_ATx(x):
        return A.T.dot(x)

    def matmat_ATsA(s):
        # A^T * Diag(s) * A
        # Для разреженных матриц это может быть оптимизировано
        if scipy.sparse.issparse(A):
            # Для разреженных матриц: A^T * (s.reshape(-1, 1) * A)
            # Это более эффективно, чем явное создание диагональной матрицы
            s_diag = scipy.sparse.diags(s)
            return A.T.dot(s_diag.dot(A))
        else:
            # Для плотных матриц
            return A.T @ (s[:, np.newaxis] * A)

    if oracle_type == 'usual':
        oracle = LogRegL2Oracle
    elif oracle_type == 'optimized':
        oracle = LogRegL2OptimizedOracle
    else:
        raise ValueError('Unknown oracle_type=%s' % oracle_type)
    
    return oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef)

#5 Реализация конечных разностей 

def grad_finite_diff(func, x, eps=1e-8):
    
    n = len(x)
    grad = np.zeros(n)
    f_x = func(x)
    
    for i in range(n):
        x_plus_eps = x.copy()
        x_plus_eps[i] += eps
        grad[i] = (func(x_plus_eps) - f_x) / eps
    
    return grad


def hess_finite_diff(func, x, eps=1e-5):
   
    n = len(x)
    hess = np.zeros((n, n))
    f_x = func(x)
    
    for i in range(n):
        x_plus_eps_i = x.copy()
        x_plus_eps_i[i] += eps
        f_x_plus_eps_i = func(x_plus_eps_i)
        
        for j in range(i, n):
            x_plus_eps_ij = x.copy()
            x_plus_eps_ij[i] += eps
            x_plus_eps_ij[j] += eps
            f_x_plus_eps_ij = func(x_plus_eps_ij)
            
            x_plus_eps_j = x.copy()
            x_plus_eps_j[j] += eps
            f_x_plus_eps_j = func(x_plus_eps_j)
            
            hess_ij = (f_x_plus_eps_ij - f_x_plus_eps_i - f_x_plus_eps_j + f_x) / (eps ** 2)
            hess[i, j] = hess_ij
            hess[j, i] = hess_ij
    
    return hess



# ## 1. Траектория градиентного спуска на квадратичной функции

import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import time
from datetime import datetime
from sklearn.datasets import load_svmlight_file

import optimization
import oracles
from plot_trajectory_2d import plot_levels, plot_trajectory


# Функция для генерации квадратичной задачи с заданным числом обусловленности
def generate_quadratic_problem(n, kappa, seed=42):
    """
    Генерирует квадратичную задачу f(x) = 1/2 x^T A x - b^T x
    с числом обусловленности kappa.
    """
    np.random.seed(seed)
    
    # Генерируем диагональные элементы в диапазоне [1, kappa]
    diag = np.random.uniform(1, kappa, n)
    # Упорядочиваем, чтобы min = 1, max = kappa
    diag = np.sort(diag)
    diag[0] = 1.0
    diag[-1] = kappa
    
    # Создаем диагональную матрицу
    A = scipy.sparse.diags(diag, format='dia')
    
    # Генерируем случайный вектор b
    b = np.random.randn(n)
    
    return A, b


# Эксперимент 1: Траектория градиентного спуска на квадратичных функциях

# Функция 1: Хорошо обусловленная (kappa = 2)
A1, b1 = generate_quadratic_problem(2, 2, seed=1)
oracle1 = oracles.QuadraticOracle(A1, b1)

# Функция 2: Плохо обусловленная (kappa = 100)
A2, b2 = generate_quadratic_problem(2, 100, seed=1)
oracle2 = oracles.QuadraticOracle(A2, b2)

# Функция 3: Очень плохо обусловленная (kappa = 1000)
A3, b3 = generate_quadratic_problem(2, 1000, seed=1)
oracle3 = oracles.QuadraticOracle(A3, b3)

# Запускаем градиентный спуск с разными стратегиями выбора шага
x0 = np.array([3.0, 2.0])
tolerance = 1e-6

# Для хорошо обусловленной функции
print("Хорошо обусловленная функция (kappa=2):")
methods = [
    {'method': 'Constant', 'c': 0.1},
    {'method': 'Constant', 'c': 0.5},
    {'method': 'Armijo', 'c1': 1e-4, 'alpha_0': 1.0},
    {'method': 'Wolfe', 'c1': 1e-4, 'c2': 0.9}
]

results1 = {}
for ls_options in methods:
    name = f"{ls_options['method']}"
    if 'c' in ls_options:
        name += f" c={ls_options['c']}"
    print(f"  {name}...")
    x_star, msg, history = optimization.gradient_descent(
        oracle1, x0, tolerance=tolerance, max_iter=1000,
        line_search_options=ls_options, trace=True, display=False
    )
    results1[name] = {'x_star': x_star, 'msg': msg, 'history': history}
    print(f"    {msg}, итераций: {len(history['func'])-1}")

# Визуализация траекторий
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Функция 1: хорошо обусловленная
ax = axes[0, 0]
plot_levels(oracle1.func, xrange=[-2, 4], yrange=[-1, 3], levels=[0, 0.5, 2, 5, 10, 20])
for name, result in results1.items():
    if result['history'] is not None and 'x' in result['history']:
        plot_trajectory(oracle1.func, result['history']['x'], label=name)
ax.set_title(f'Хорошо обусловленная (kappa=2), итераций: {[len(r["history"]["func"])-1 for r in results1.values()]}')
ax.legend()
ax.grid(True)

# Функция 2: плохо обусловленная
ax = axes[0, 1]
plot_levels(oracle2.func, xrange=[-2, 4], yrange=[-1, 3], levels=[0, 0.5, 2, 5, 10, 20])
results2 = {}
for ls_options in methods:
    name = f"{ls_options['method']}"
    if 'c' in ls_options:
        name += f" c={ls_options['c']}"
    x_star, msg, history = optimization.gradient_descent(
        oracle2, x0, tolerance=tolerance, max_iter=1000,
        line_search_options=ls_options, trace=True, display=False
    )
    results2[name] = {'x_star': x_star, 'msg': msg, 'history': history}
    if history is not None and 'x' in history:
        plot_trajectory(oracle2.func, history['x'], label=name)
ax.set_title(f'Плохо обусловленная (kappa=100), итераций: {[len(r["history"]["func"])-1 for r in results2.values()]}')
ax.legend()
ax.grid(True)

# Функция 3: очень плохо обусловленная
ax = axes[1, 0]
plot_levels(oracle3.func, xrange=[-2, 4], yrange=[-1, 3], levels=[0, 0.5, 2, 5, 10, 20])
results3 = {}
for ls_options in methods:
    name = f"{ls_options['method']}"
    if 'c' in ls_options:
        name += f" c={ls_options['c']}"
    x_star, msg, history = optimization.gradient_descent(
        oracle3, x0, tolerance=tolerance, max_iter=1000,
        line_search_options=ls_options, trace=True, display=False
    )
    results3[name] = {'x_star': x_star, 'msg': msg, 'history': history}
    if history is not None and 'x' in history:
        plot_trajectory(oracle3.func, history['x'], label=name)
ax.set_title(f'Очень плохо обусловленная (kappa=1000), итераций: {[len(r["history"]["func"])-1 for r in results3.values()]}')
ax.legend()
ax.grid(True)

# Сходимость по функции для разных стратегий на плохо обусловленной функции
ax = axes[1, 1]
for name, result in results2.items():
    if result['history'] is not None:
        iterations = range(len(result['history']['func']))
        ax.semilogy(iterations, result['history']['func'], label=name, linewidth=2)
ax.set_xlabel('Итерация')
ax.set_ylabel('f(x_k)')
ax.set_title('Сходимость по функции (kappa=100)')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.savefig('trajectory_experiment.png', dpi=150)
plt.show()

# Выводы по эксперименту 1
print("""
Выводы по эксперименту 1:
1. Число обусловленности сильно влияет на скорость сходимости градиентного спуска.
   Чем хуже обусловленность, тем больше итераций требуется для сходимости.
2. Константный шаг критически зависит от выбора параметра. Слишком большой шаг
   приводит к расходимости, слишком маленький - к медленной сходимости.
3. Адаптивные методы (Armijo, Wolfe) автоматически подбирают подходящий шаг и
   работают хорошо для всех функций.
4. Wolfe условия обычно дают более быструю сходимость, чем Armijo, но требуют
   больше вычислений на итерацию.
""")

# ## 2. Зависимость числа итераций от числа обусловленности и размерности

def run_conditioning_experiment():
    """
    Эксперимент по исследованию зависимости числа итераций от числа обусловленности
    и размерности пространства.
    """
    np.random.seed(42)
    
    # Размерности для тестирования
    dimensions = [10, 100, 500, 1000]
    # Числа обусловленности
    kappas = np.logspace(1, 4, 10)  # от 10 до 10000
    
    # Параметры эксперимента
    tolerance = 1e-6
    n_trials = 5  # число запусков для каждого параметра
    
    results = {}
    colors = ['red', 'blue', 'green', 'orange']
    
    for dim_idx, n in enumerate(dimensions):
        print(f"\nРазмерность n = {n}")
        color = colors[dim_idx % len(colors)]
        
        iterations_matrix = []
        
        for kappa in kappas:
            kappa_iterations = []
            
            for trial in range(n_trials):
                # Генерируем задачу
                A, b = generate_quadratic_problem(n, kappa, seed=42 + trial * 100 + dim_idx * 10)
                oracle = oracles.QuadraticOracle(A, b)
                
                # Начальная точка
                x0 = np.random.randn(n)
                
                # Запускаем градиентный спуск с Armijo правилом
                x_star, msg, history = optimization.gradient_descent(
                    oracle, x0, tolerance=tolerance, max_iter=10000,
                    line_search_options={'method': 'Armijo', 'c1': 1e-4, 'alpha_0': 1.0},
                    trace=False, display=False
                )
                
                if msg == 'success':
                    iterations = len(history['func']) - 1 if history else 0
                else:
                    iterations = 10000  # максимум
                
                kappa_iterations.append(iterations)
                print(f"  kappa = {kappa:.1f}, trial {trial+1}: {iterations} итераций")
            
            iterations_matrix.append(kappa_iterations)
        
        results[n] = {'kappas': kappas, 'iterations': iterations_matrix, 'color': color}
    
    return results

# Запускаем эксперимент (может занять много времени)
# Для демонстрации используем меньшие размерности
results = run_conditioning_experiment()

# Визуализация результатов
plt.figure(figsize=(12, 8))

for n, data in results.items():
    kappas = data['kappas']
    iterations_matrix = np.array(data['iterations'])
    color = data['color']
    
    # Рисуем все кривые для данного n
    for trial in range(iterations_matrix.shape[1]):
        plt.loglog(kappas, iterations_matrix[:, trial], 
                   color=color, alpha=0.3, linewidth=1)
    
    # Рисуем среднее значение жирной линией
    mean_iterations = np.mean(iterations_matrix, axis=1)
    plt.loglog(kappas, mean_iterations, color=color, linewidth=3, 
               label=f'n = {n} (среднее)')

plt.xlabel('Число обусловленности κ')
plt.ylabel('Число итераций до сходимости')
plt.title('Зависимость числа итераций градиентного спуска от κ и n')
plt.grid(True, which='both', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('conditioning_experiment.png', dpi=150)
plt.show()

print("""
Выводы по эксперименту 2:
1. Число итераций градиентного спуска линейно растет с числом обусловленности κ.
   Это соответствует теоретической оценке O(κ log(1/ε)).
2. Размерность n практически не влияет на число итераций для квадратичных задач
   с диагональной матрицей Гессе, так как в этом случае градиентный спуск
   сводится к независимой оптимизации по каждой координате.
3. Разброс результатов обусловлен случайной генерацией матрицы A и начальной точки.
4. Адаптивные методы выбора шага (Armijo) позволяют достичь теоретической
   линейной зависимости от κ без подбора параметров.
""")

# ## 3. Сравнение градиентного спуска и метода Ньютона на задаче логистической регрессии

def load_libsvm_data(dataset_name):
    """
    Загрузка данных с сайта LIBSVM.
    Для демонстрации используем локальные файлы или генерируем синтетические данные.
    """
    try:
        if dataset_name == 'w8a':
            X, y = load_svmlight_file('data/w8a')
        elif dataset_name == 'gisette':
            X, y = load_svmlight_file('data/gisette_scale')
        elif dataset_name == 'real-sim':
            X, y = load_svmlight_file('data/real-sim')
        else:
            raise ValueError(f"Unknown dataset: {dataset_name}")
        y = 2 * (y > 0) - 1  # Преобразуем в {-1, 1}
        return X, y
    except:
        # Если файлы не найдены, генерируем синтетические данные
        print(f"Датасет {dataset_name} не найден, генерируем синтетические данные")
        np.random.seed(42)
        m, n = 1000, 100
        X = np.random.randn(m, n)
        y = np.sign(np.random.randn(m))
        return X, y

def run_logreg_comparison(dataset_name):
    """
    Сравнение градиентного спуска и метода Ньютона на задаче логистической регрессии.
    """
    print(f"\n=== Сравнение на датасете {dataset_name} ===")
    
    # Загрузка данных
    X, y = load_libsvm_data(dataset_name)
    m, n = X.shape
    regcoef = 1.0 / m
    
    print(f"Размерность: m={m}, n={n}")
    print(f"Коэффициент регуляризации: {regcoef:.6f}")
    
    # Создание оракула
    oracle = oracles.create_log_reg_oracle(X, y, regcoef, oracle_type='usual')
    
    # Начальная точка
    x0 = np.zeros(n)
    tolerance = 1e-6
    
    # Градиентный спуск
    print("\nЗапуск градиентного спуска...")
    start_time = time.time()
    x_gd, msg_gd, history_gd = optimization.gradient_descent(
        oracle, x0, tolerance=tolerance, max_iter=1000,
        line_search_options={'method': 'Armijo', 'c1': 1e-4, 'alpha_0': 1.0},
        trace=True, display=False
    )
    gd_time = time.time() - start_time
    print(f"Градиентный спуск: {msg_gd}, итераций: {len(history_gd['func'])-1}, время: {gd_time:.2f}с")
    
    # Метод Ньютона
    print("\nЗапуск метода Ньютона...")
    start_time = time.time()
    x_newton, msg_newton, history_newton = optimization.newton(
        oracle, x0, tolerance=tolerance, max_iter=100,
        line_search_options={'method': 'Wolfe', 'c1': 1e-4, 'c2': 0.9},
        trace=True, display=False
    )
    newton_time = time.time() - start_time
    print(f"Метод Ньютона: {msg_newton}, итераций: {len(history_newton['func'])-1}, время: {newton_time:.2f}с")
    
    return {
        'dataset': dataset_name,
        'm': m, 'n': n,
        'gd': {'history': history_gd, 'time': gd_time, 'msg': msg_gd},
        'newton': {'history': history_newton, 'time': newton_time, 'msg': msg_newton}
    }

# Запускаем сравнение на нескольких датасетах
datasets = ['w8a', 'gisette', 'real-sim']
results = {}

for dataset in datasets:
    try:
        results[dataset] = run_logreg_comparison(dataset)
    except Exception as e:
        print(f"Ошибка при обработке {dataset}: {e}")

# Визуализация результатов
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, (dataset, result) in enumerate(results.items()):
    # График 1: Значение функции от времени
    ax = axes[0, idx]
    
    if result['gd']['history']:
        time_gd = result['gd']['history']['time']
        func_gd = result['gd']['history']['func']
        ax.semilogy(time_gd, func_gd, 'b-', linewidth=2, label='GD')
    
    if result['newton']['history']:
        time_newton = result['newton']['history']['time']
        func_newton = result['newton']['history']['func']
        ax.semilogy(time_newton, func_newton, 'r-', linewidth=2, label='Newton')
    
    ax.set_xlabel('Время (с)')
    ax.set_ylabel('f(x_k)')
    ax.set_title(f'{dataset}: Значение функции\nm={result["m"]}, n={result["n"]}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # График 2: Относительная норма градиента от времени
    ax = axes[1, idx]
    
    if result['gd']['history']:
        time_gd = result['gd']['history']['time']
        grad_norm_gd = result['gd']['history']['grad_norm']
        grad_norm_0 = grad_norm_gd[0] if grad_norm_gd else 1
        rel_grad_gd = [gn**2 / grad_norm_0**2 for gn in grad_norm_gd]
        ax.semilogy(time_gd, rel_grad_gd, 'b-', linewidth=2, label='GD')
    
    if result['newton']['history']:
        time_newton = result['newton']['history']['time']
        grad_norm_newton = result['newton']['history']['grad_norm']
        grad_norm_0 = grad_norm_newton[0] if grad_norm_newton else 1
        rel_grad_newton = [gn**2 / grad_norm_0**2 for gn in grad_norm_newton]
        ax.semilogy(time_newton, rel_grad_newton, 'r-', linewidth=2, label='Newton')
    
    ax.set_xlabel('Время (с)')
    ax.set_ylabel('||∇f(x_k)||² / ||∇f(x_0)||²')
    ax.set_title(f'{dataset}: Относительная норма градиента')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('logreg_comparison.png', dpi=150)
plt.show()

# Анализ сложности методов
print("""
Анализ сложности методов:

Градиентный спуск:
- Стоимость одной итерации: O(m * n) - вычисление градиента (A^T * (b * sigma(-b*Ax)))
- Память: O(n) - хранение вектора x и градиента
- Сходимость: линейная, O(κ log(1/ε)) итераций

Метод Ньютона:
- Стоимость одной итерации: O(n^3) - решение системы с матрицей Гессе
- Память: O(n^2) - хранение матрицы Гессе
- Сходимость: квадратичная вблизи решения, O(log log(1/ε)) итераций

Выводы по эксперименту 3:
1. Для задач малой размерности (n < 1000) метод Ньютона сходится за 
   значительно меньшее число итераций и часто быстрее по времени.
2. Для задач большой размерности (n > 1000) градиентный спуск может быть 
   предпочтительнее из-за низкой стоимости итерации O(m*n) против O(n^3).
3. Метод Ньютона требует хранения матрицы Гессе O(n^2), что может быть 
   проблематично для задач с n > 10^5.
4. Выбор метода зависит от соотношения m и n, а также от требуемой точности.
""")

# ## Проверка с помощью presubmit_tests.py

# Запуск тестов
!nosetests3 presubmit_tests.py

<class 'SyntaxError'>: invalid syntax (1210464727.py, line 579)